# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path = "../weather_data.csv"

weather_data = pd.read_csv(path)
weather_data

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Wind Speed,Cloudiness,Country,Data Receiving Time (GMT)
0,0,juuru,59.06,24.96,48.20,100,9.17,90,EE,1603732219
1,1,jamestown,42.10,-79.24,48.99,100,3.36,90,US,1603732414
2,2,soure,-0.72,-48.52,87.40,61,12.37,0,BR,1603732220
3,3,kahului,20.89,-156.47,75.20,69,6.76,20,US,1603731829
4,4,chuy,-33.70,-53.46,60.67,95,14.47,100,UY,1603731897
...,...,...,...,...,...,...,...,...,...,...
567,567,alofi,-19.06,-169.92,74.66,84,22.82,99,NU,1603731964
568,568,tsabong,-26.02,22.40,88.39,21,11.92,32,BW,1603732271
569,569,talara,-4.58,-81.27,77.00,65,11.41,0,PE,1603731997
570,570,pilna,55.55,45.92,31.93,95,4.21,42,RU,1603732271


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
gmaps.configure(api_key=g_key)

location = weather_data[["Latitude", "Longitude"]].astype(float)

humidity = weather_data['Humidity']

In [26]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1.4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [42]:
hotel_df = weather_data.loc[(weather_data['Max Temperature'] < 80) &
                                (weather_data['Max Temperature'] > 70) &
                                (weather_data['Wind Speed'] < 5) &
                                (weather_data['Cloudiness'] == 0) &
                                (weather_data['Humidity'] < 90)]
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Wind Speed,Cloudiness,Country,Data Receiving Time (GMT)
52,52,dondaicha,21.33,74.57,79.30,43,4.07,0,IN,1603732224
112,112,saint-philippe,-21.36,55.77,71.60,83,3.36,0,RE,1603732445
166,166,bela,25.93,81.98,75.38,30,4.32,0,IN,1603732233
343,343,general roca,-39.03,-67.58,73.40,28,4.70,0,AR,1603732249
432,432,naspur,18.83,79.45,74.91,70,3.38,0,IN,1603732259
460,460,sohagpur,22.70,78.20,71.67,46,3.09,0,IN,1603732261
464,464,riyadh,24.69,46.72,75.20,27,3.36,0,SA,1603731769
493,493,shahjahanpur,27.88,79.92,73.04,25,4.59,0,IN,1603732264
557,557,allapalli,19.42,80.07,70.81,77,1.48,0,IN,1603732270


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [43]:
hotel_df['Hotel Name'] = ""

params = {
    "radius": 5000,
    "types": "lodging",
    "language": "en",
    "key": g_key
}
    
for index, row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    lng = row["Longitude"]

    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except:
        print("Missing field/result... skipping.")
hotel_df

C:\Users\austi\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\austi\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Wind Speed,Cloudiness,Country,Data Receiving Time (GMT),Hotel Name
52,52,dondaicha,21.33,74.57,79.30,43,4.07,0,IN,1603732224,Mahavir Farm House
112,112,saint-philippe,-21.36,55.77,71.60,83,3.36,0,RE,1603732445,"Chambres d'hôte ""La Trinité"""
166,166,bela,25.93,81.98,75.38,30,4.32,0,IN,1603732233,Siddharth Hotel
343,343,general roca,-39.03,-67.58,73.40,28,4.70,0,AR,1603732249,Hotel El Recreo
432,432,naspur,18.83,79.45,74.91,70,3.38,0,IN,1603732259,Mancherla venkatswamy
460,460,sohagpur,22.70,78.20,71.67,46,3.09,0,IN,1603732261,Hotel Vatika Inn
464,464,riyadh,24.69,46.72,75.20,27,3.36,0,SA,1603731769,InterContinental Riyadh
493,493,shahjahanpur,27.88,79.92,73.04,25,4.59,0,IN,1603732264,OYO 37782 Hotel Shree Mahakali Palace
557,557,allapalli,19.42,80.07,70.81,77,1.48,0,IN,1603732270,Hotel IK Residency


In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [45]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1.4)
hotel_layer = gmaps.marker_layer(
    locations, info_box_content= hotel_info
)


fig.add_layer(hotel_layer)

fig.add_layer(heat_layer)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))